# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> _Colab Notebook Template_

## Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.8.2 on the current Colab Runtime...
2022-11-02 02:02:18 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.8/julia-1.8.2-linux-x86_64.tar.gz [135859273/135859273] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.8

Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then
jump to the 'Checking the Installation' section.


# Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [2]:
versioninfo()

Julia Version 1.11.5
Commit 760b2e5b739 (2025-04-14 06:53 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × AMD EPYC 7B12
  WORD_SIZE: 64
  LLVM: libLLVM-16.0.6 (ORCJIT, znver2)
Threads: 2 default, 0 interactive, 1 GC (on 2 virtual cores)
Environment:
  LD_LIBRARY_PATH = /usr/local/nvidia/lib:/usr/local/nvidia/lib64
  JULIA_NUM_THREADS = auto


In [3]:
using BenchmarkTools

M = rand(2^11, 2^11)

@btime $M * $M;

LoadError: ArgumentError: Package BenchmarkTools not found in current path.
- Run `import Pkg; Pkg.add("BenchmarkTools")` to install the BenchmarkTools package.

In [4]:
try
    using CUDA
catch
    println("No GPU found.")
else
    run(`nvidia-smi`)
    # Create a new random matrix directly on the GPU:
    M_on_gpu = CUDA.CURAND.rand(2^11, 2^11)
    @btime $M_on_gpu * $M_on_gpu; nothing
end

LoadError: LoadError: UndefVarError: `@btime` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
in expression starting at In[4]:9

# Need Help?

* Learning: https://julialang.org/learning/
* Documentation: https://docs.julialang.org/
* Questions & Discussions:
  * https://discourse.julialang.org/
  * http://julialang.slack.com/
  * https://stackoverflow.com/questions/tagged/julia

If you ever ask for help or file an issue about Julia, you should generally provide the output of `versioninfo()`.

Add new code cells by clicking the `+ Code` button (or _Insert_ > _Code cell_).

Have fun!

<img src="https://raw.githubusercontent.com/JuliaLang/julia-logo-graphics/master/images/julia-logo-mask.png" height="100" />

In [6]:
println("Hello from Julia!")

Hello from Julia!


In [7]:
# Q6_analysis.jl
# This script defines the Q6 polyhedron (Mx >= 1, x >= 0) WITH upper bounds,
# finds its extreme points and rays, and then analyzes a subset of extreme points.

import Pkg
Pkg.add("Polymake")

using Polymake
using LinearAlgebra # Needed for identity matrix (LinearAlgebra.I)

# --- 1. Define the Polytope's H-representation (Ax <= b format) ---

# M matrix as given
M_matrix = Polymake.Matrix([
    1 0 1 0 1 0; # Use spaces for columns, semicolon for new row
    1 0 0 1 0 1;
    0 1 1 0 0 1;
    0 1 0 1 1 0   # No semicolon on the last row
])

# Convert Mx >= 1 to -Mx <= -1
A_ub_Mx = -M_matrix
b_ub_Mx = Polymake.Vector([-1, -1, -1, -1])

# Non-negativity constraints: xj >= 0  => -xj <= 0
A_ub_nonneg = -Polymake.Matrix(Matrix(LinearAlgebra.I, 6, 6))
b_ub_nonneg = Polymake.Vector([0, 0, 0, 0, 0, 0])

# Upper bounds xj <= 1  => xj <= 1
A_ub_upper = Polymake.Matrix(Matrix(LinearAlgebra.I, 6, 6))
b_ub_upper = Polymake.Vector([1, 1, 1, 1, 1, 1])

# Combine all constraints (using Polymake's vcat for vertical concatenation)
A_ub_combined = Polymake.vcat(A_ub_Mx, A_ub_nonneg, A_ub_upper)
b_ub_combined = Polymake.vcat(b_ub_Mx, b_ub_nonneg, b_ub_upper)

# --- Construct the Polyhedron in Polymake ---
# Polymake Polyhedron (note: not Polytope) takes INEQUALITIES
# Combine b and A into a single matrix for Polymake.
h_matrix = Polymake.hcat(b_ub_combined, A_ub_combined)

println("\n--- Polytope Definition (in Polymake H-matrix format) ---")
println("H-matrix rows for [b | A] for Ax <= b:\n", h_matrix)

# Create the Polyhedron object (CORRECTED: Use Polyhedron constructor)
P = Polymake.Polyhedron(INEQUALITIES=h_matrix) # <-- CORRECTED LINE

# --- 1. Find ALL Extreme Points (Vertices) of the Polytope ---
println("\n--- Computing All Extreme Points (Vertices) ---")
# Accessing properties like VERTICES forces computation
# Check if it's bounded first, though with upper bounds it should be
if P.IS_BOUNDED # Check if the constructed polyhedron is bounded
    vertices = P.VERTICES
    println("Polymake.Vector objects for vertices:\n", vertices)

    # Convert to standard Julia Array of Floats for easier handling and display
    all_vertices_julia = [Float64.(v) for v in vertices]
    println("All Vertices (Julia Array, rounded for display):\n")
    for i in 1:length(all_vertices_julia)
        println("  Vertex ", i, ": ", round.(all_vertices_julia[i], digits=6))
    end
    println(string("Found ", length(all_vertices_julia), " extreme points (vertices)."))
else
    println("The polyhedron is unbounded, so compute_polytope_vertices is not applicable.")
    println("Access P.RAYS for extreme rays if needed.")
    all_vertices_julia = [] # Ensure it's empty if unbounded
end


# Exit if no vertices are found, as subsequent steps depend on them
if isempty(all_vertices_julia)
    println("Cannot proceed with subset analysis if no finite vertices are found.")
    exit()
end

# --- 2. EXAMPLE: Select a Subset of Extreme Points ---
# Choose a subset from the computed vertices.
# For demonstration, let's pick the first two vertices found by Polymake (adjust as needed).
if length(all_vertices_julia) >= 2
    subset_points = [all_vertices_julia[1], all_vertices_julia[2]] # Julia uses 1-based indexing
    println("\n--- Example Subset of Extreme Points ---")
    println("Subset Points:\n", subset_points)
else
    println("\n--- Not enough vertices to form a subset example. Using first vertex. ---")
    subset_points = [all_vertices_julia[1]] # Just use the first one if only one exists
end

# --- 3. Identify the Smallest Face the Subset Resides On ---
println("\n--- Identifying Smallest Face ---")

# To use FACE_FROM_POINTS, we need the 1-based indices of the subset points within P.VERTICES.
subset_indices = Int[] # Initialize an empty array to store integer indices
for p_subset in subset_points
    for (idx, p_all) in enumerate(all_vertices_julia)
        # Compare points using a small tolerance for floating point equality
        if all(abs.(p_subset - p_all) .< 1e-9)
            push!(subset_indices, idx) # Add the 1-based index
            break # Found it, move to next subset point
        end
    end
end

if isempty(subset_indices)
    println("Error: Subset points not found among the computed vertices.")
else
    println("Indices of subset points in P.VERTICES (1-based): ", subset_indices)

    # Get the face object from Polymake using its indices
    F = P.FACE_FROM_POINTS(subset_indices)
    println("Smallest face containing the subset (as a Polymake Polytope object): ", F) # F is a Polyhedron/Polytope object

    # --- 4. Calculate the Dimension of That Face ---
    println("\n--- Calculating Face Dimension ---")
    face_dimension = F.DIM # Access the dimension property of the face object
    println("The dimension of the smallest face containing the subset is: ", face_dimension)
end

# --- 5. Access the Face Lattice (More Advanced) ---
println("\n--- Accessing Face Lattice (Example) ---")
# This can be computationally intensive for high dimensions/many vertices.
# lattice = P.FACE_LATTICE # Uncomment with caution for very large polyhedra
# println("Face Lattice (Polymake Graph object):", lattice)
# println("Number of faces in lattice: ", lattice.N_NODES)

println("\n--- Analysis Complete ---")

LoadError: ArgumentError: Package Polymake not found in current path.
- Run `import Pkg; Pkg.add("Polymake")` to install the Polymake package.

In [ ]:
import Pkg
Pkg.add("Polymake")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed snappy_jll ─────────────── v1.2.3+0
   Installed OpenBLAS32_jll ─────────── v0.3.29+0
   Installed lib4ti2_jll ────────────── v1.6.10+0
   Installed FLINT_jll ──────────────── v301.300.101+0
   Installed TOPCOM_jll ─────────────── v0.17.8+0
   Installed MongoC_jll ─────────────── v1.28.1+0
   Installed Perl_jll ───────────────── v5.34.1+0
   Installed boost_jll ──────────────── v1.87.0+0
   Installed Hwloc_jll ──────────────── v2.12.1+0
   Installed Mongoc ─────────────────── v0.9.2
   Installed MUMPS_seq_jll ──────────── v500.800.0+0
   Installed GLPK_jll ───────────────── v5.0.1+1
   Installed nauty_jll ──────────────── v2.8.10+1
   Installed polymake_jll ───────────── v400.1400.0+0
   Installed BinaryWrappers ─────────── v0.1.3
   Installed Ncurses_jll ────────────── v6.5.1+0
   Installed libpolymake_julia_jll ──── v0.14.0+0
   Installed METIS_jll ──────────────── v5.1.3+0
   In